# Network Graphing

Using networkx to create a network of JQA papers. This notebook follows and copies much from the following tutorial. If this is helpful, cite them:


John R. Ladd, Jessica Otis, Christopher N. Warren, and Scott Weingart, "Exploring and Analyzing Network Data with Python," The Programming Historian 6 (2017), https://doi.org/10.46430/phen0064.

In [20]:
import re
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import community
from operator import itemgetter

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/SemanticData/"

## Import Data & Create Graph Object

In [15]:
%%time

# Read in nodes and edges.
nodes = pd.read_csv(abs_dir + "Output/Dataframes/Graphs/JQA_Network_correlation/graphLasso_nodes.csv",
                    sep = ',')

edges = pd.read_csv(abs_dir + "Output/Dataframes/Graphs/JQA_Network_correlation/graphLasso_links.csv",
                    sep = ',')

# Create dictionary to map values to codes.
nodes_dictionary = nodes['label'].to_dict()

# Map labels back onto source and target.
edges = edges.replace({'source':nodes_dictionary, 'target':nodes_dictionary})

# Convert edges dataframe to edges tuple (compatible with graph object below).
edges = [tuple(x) for x in edges[['source', 'target']].to_numpy()]

# Initialize graph object.
G = nx.Graph()

# Add nodes and edges to graph object.
G.add_nodes_from(nodes['label'])
G.add_edges_from(edges)

print (nx.info(G))

Name: 
Type: Graph
Number of nodes: 5135
Number of edges: 4406
Average degree:   1.7161
CPU times: user 555 ms, sys: 12.4 ms, total: 568 ms
Wall time: 572 ms


## Network Measurements

### Network Density

Network density is measured on a scale from 0 (sparse) to 1 (dense). From Ladd et. al, 

    "A 0 would mean that there are no connections at all, and a 1 would indicate that all 
    possible edges are present (a perfectly connected network)"

#### Subcomponents
Subgraphs (sub-components of a graph) can express the density of networks that are not connected or have decentralized structures.

#### Triadic Closure
Similarly, triadic closure articulates network interconnectedness but through inferred connections (if one entity relates to two other entities, who are not directly related, then it can be inferred that all three share a relationship). Triadic closure (transitivity) is the ratio of all actual connections over possible connections (nodes^2?).

In [18]:
%%time

# Measure network density.
density = nx.density(G)
print ("Network density:", density)

# Related to diameter, check if network is connected and, therefore, can have a diameter.
print ("Is the network connected?", nx.is_connected(G))

# Get a list of network components (communities).
# Find the largest component.
components = nx.connected_components(G)
largest_component = max(components, key = len)

# Create a subgraph of the largest component and measure its diameter.
subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print ("Network diameter of the largest component:", diameter)

# Find triadic closure (similar to density).
triadic_closure = nx.transitivity(G)
print ("Triadic closure:", triadic_closure)

Network density: 0.0003342552030130004
Is the network connected? False
Network diameter of the largest component: 1
Triadic closure: 1.0
CPU times: user 83.9 ms, sys: 2.37 ms, total: 86.2 ms
Wall time: 85.4 ms


### Centrality

Centrality refers to various methods used to determine the importance of nodes in a graph.

#### Degree
One measure of centrality is to find each node's degree. The degree of a node is the sum of all its edges. If a node links to three other entities, then its degree is three. A node with a high degree is considered a hub or spoke.

In [22]:
%%time

# Create a dictionary of each nodes degree.
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

# Sort nodes by degree and print top results.
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print ("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print (d)

Top 20 nodes by degree:
('bonaparte-charles', 12)
('browne-peter', 12)
('calvert-george3', 12)
('gerard-conrad', 12)
('haviland-john', 12)
('kneller-godfrey', 12)
('mcneal-unknown2', 12)
('peale-charles', 12)
('peale-titian', 12)
('peters-unknown4', 12)
('philippon-unknown', 12)
('vandyke-anthony', 12)
('wheldon-wilmon', 12)
('caton-unknown', 11)
('clapp-asa', 11)
('elder-unknown', 11)
('hone-unknown2', 11)
('moseby-unknown', 11)
('oliver-unknown', 11)
('proctor-unkown', 11)
CPU times: user 6.35 ms, sys: 295 µs, total: 6.65 ms
Wall time: 6.52 ms


#### Betweenness Centrality

While degree privileges the nodes with the most direct connections, some nodes might be important because they glue together two different groups. These broker nodes are important because they keep different branches of a network attached.

In [27]:
%%time

# Get centrality measurements.
betweenness_dict = nx.betweenness_centrality(G)

# Assign each measuremnt to attributes within the network.
nx.set_node_attributes(G, betweenness_dict, "betweenness")

# Sort and print nodes with high centrality scores.
sorted_betweenness = sorted(betweenness_dict.items(), key = itemgetter(1), reverse = True)

print ("Top 20 nodes by betweenness centrality")
for b in sorted_betweenness[:20]:
    print (b)

Top 20 nodes by betweenness centrality
('Ishbosheth', 0.0)
('Willis Alston', 0.0)
('abbot-benjamin', 0.0)
('abbot-joel', 0.0)
('abbot-joel2', 0.0)
('abbot-joseph', 0.0)
('abbot-unknown', 0.0)
('abbott-joel', 0.0)
('abbott-joseph', 0.0)
('abdon', 0.0)
('abner', 0.0)
('abraham', 0.0)
('abram-unknown', 0.0)
('absalom', 0.0)
('achish', 0.0)
('adadms-john', 0.0)
('adair-robert', 0.0)
('adams-abigail', 0.0)
('adams-abigail2', 0.0)
('adams-abigail3', 0.0)
CPU times: user 11.5 s, sys: 54.3 ms, total: 11.6 s
Wall time: 11.7 s


#### Eigenvector Centrality

Eigenvector centrality, like degree, considers centrality an aspect of interconnectedness. It also counts node degree. Eigenvector centrality, however, goes a step further and also considers whether a hub is connected to other hubs.

In [28]:
%%time

# Get centrality measurements.
eigenvector_dict = nx.eigenvector_centrality(G)

# Assign each measuremnt to attributes within the network.
nx.set_node_attributes(G, eigenvector_dict, "eigenvector")

# Sort and print nodes with high centrality scores.
sorted_eigenvector = sorted(eigenvector_dict.items(), key = itemgetter(1), reverse = True)

print ("Top 20 nodes by eigenvector centrality")
for b in sorted_eigenvector[:20]:
    print (b)

Top 20 nodes by eigenvector centrality
('bonaparte-charles', 0.27731903930585705)
('browne-peter', 0.27731903930585705)
('calvert-george3', 0.27731903930585705)
('gerard-conrad', 0.27731903930585705)
('haviland-john', 0.27731903930585705)
('kneller-godfrey', 0.27731903930585705)
('mcneal-unknown2', 0.27731903930585705)
('peale-charles', 0.27731903930585705)
('peale-titian', 0.27731903930585705)
('peters-unknown4', 0.27731903930585705)
('philippon-unknown', 0.27731903930585705)
('vandyke-anthony', 0.27731903930585705)
('wheldon-wilmon', 0.27731903930585705)
('caton-unknown', 0.004318671409409538)
('clapp-asa', 0.004318671409409538)
('elder-unknown', 0.004318671409409538)
('hone-unknown2', 0.004318671409409538)
('moseby-unknown', 0.004318671409409538)
('oliver-unknown', 0.004318671409409538)
('proctor-unkown', 0.004318671409409538)
CPU times: user 360 ms, sys: 3.38 ms, total: 363 ms
Wall time: 365 ms


#### Combining Measurements

Often nodes with high degrees will also have high eigenvector or betweenness centrality. But, what if you want to know which of the high eigenvector centrality nodes had low degree?

In [29]:
%%time

# Create list of top 20 nodes by eigenvector.
top_eigenvector = sorted_eigenvector[:20]

# Find and print their degrees.
for te in top_eigenvector:
    degree = degree_dict[te[0]]
    print ("Name:", te[0], "| Eigenvector Centrality:", te[1], "| Degree:", degree)

Name: bonaparte-charles | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: browne-peter | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: calvert-george3 | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: gerard-conrad | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: haviland-john | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: kneller-godfrey | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: mcneal-unknown2 | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: peale-charles | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: peale-titian | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: peters-unknown4 | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: philippon-unknown | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: vandyke-anthony | Eigenvector Centrality: 0.27731903930585705 | Degree: 12
Name: wheldon-wilmon | Eigenvector C

## Community Detection

Community refers to the subgroups that appear within a network.

#### Modularity

Modularity helps discover sub-communities within a network by searching for pockets of nodes with a relatively high density.

In [30]:
%%time

# Find communities.
communities = community.greedy_modularity_communities(G)

# Create a dictionary that maps nodes to their community.
modularity_dict = {}
for i, c in enumerate(communities):
    for name in c:
        modularity_dict[name] = i
        
# Add modularity information to graph object.
nx.set_node_attributes(G, modularity_dict, 'modularity')

CPU times: user 182 ms, sys: 4.09 ms, total: 186 ms
Wall time: 187 ms


#### Combining Methods

Combining eigenvector centrality as ranking indicates nodes central to each community.

In [33]:
%%time

# Examine nodes from first modularity class.
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Create dictionary of eigenvector centralities of those nodes.
class0_eig = {n:G.nodes[n]['eigenvector'] for n in class0}

# Sort and print first 5 results.
class0_sorted_eig = sorted(class0_eig.items(), key = itemgetter(1), reverse = True)

print ("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_eig[:5]:
    print ("Name:", node[0], "| Eigenvector Centrality:", node[1])

Modularity Class 0 Sorted by Eigenvector Centrality:
Name: bonaparte-charles | Eigenvector Centrality: 0.27731903930585705
Name: browne-peter | Eigenvector Centrality: 0.27731903930585705
Name: calvert-george3 | Eigenvector Centrality: 0.27731903930585705
Name: gerard-conrad | Eigenvector Centrality: 0.27731903930585705
Name: haviland-john | Eigenvector Centrality: 0.27731903930585705
CPU times: user 4.48 ms, sys: 975 µs, total: 5.46 ms
Wall time: 4.65 ms


#### Community Membership

In [35]:
%%time

for i, c in enumerate(communities):
    if len(c) > 2:
        print ('Class ' + str(i) + ':', list(c), '\n')

Class 0: ['kneller-godfrey', 'bonaparte-charles', 'calvert-george3', 'browne-peter', 'peters-unknown4', 'gerard-conrad', 'peale-charles', 'vandyke-anthony', 'wheldon-wilmon', 'peale-titian', 'haviland-john', 'mcneal-unknown2', 'philippon-unknown'] 

Class 1: ['rogers-moulton', 'hone-unknown2', 'tyson-unknown', 'proctor-unkown', 'caton-unknown', 'moseby-unknown', 'thompson-unknown8', 'oliver-unknown', 'randall-unknown4', 'elder-unknown', 'scott-unknown7', 'clapp-asa'] 

Class 2: ['pickman-elizabeth', 'pickman-martha', 'brazer-unknown2', 'rogers-unknown3', 'rogers-unknown2', 'felt-abigail', 'leslie-alexander', 'pickman-anstiss', 'endicott-john', 'adadms-john', 'felt-joseph'] 

Class 3: ['ellery-abraham', 'magee-unknown', 'dowse-unknown2', 'ellery-sarah', 'hubbard-unknown4', 'ellery-unknown2', 'ellery-unknown3', 'hubbard-unknown5', 'dowse-unknown', 'ellery-martha', 'shaw-unknown2'] 

Class 4: ['bethune-george', 'adams-abigail3', 'russell-catherine', 'tucker-unknown4', 'bethune-unknown', '

## Export graph object into Gephi's GEXF format.

In [36]:
%%time

nx.write_gexf(G, 'jqa_papers.gexf')

CPU times: user 260 ms, sys: 8.21 ms, total: 268 ms
Wall time: 269 ms
